# Multi-Layer Perceptron (MLP)
In questo notebook addestriamo un modello MLP con dati sintetici per un problema di classificazione binaria. Dato un dataset etichettato di dati sintetici, addestrizmo un modello MLP per trovare un confine decisionale tra due classi NON linearmente separabili.
Abbiamo provato ad eseguire lo stesso task con Logistic Regression (laboratorio [Logistic Regression](../05-LogisticRegression/)). Ma per riuscirci abbiamo dovuto ricorrere a features polinomiali.

Il vantaggio delle reti neurali come MLP e' che possono risolvere problemi complessi richiedendo una preparazione meno laboriosa dei dati.

In [ ]:
# Author: Roberto Doriguzzi-Corin
# Project: Corso di Algoritmi di Machine Learning per la rilevazione di attacchi informatici
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Import necessary libraries

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles
from sklearn.datasets import make_classification
from sklearn.metrics import classification_report, f1_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Input
import time

SEED = 1

# Create a synthetic dataset with two classes that are not linearly separable
X, y = make_circles(n_samples=1000, noise=0.1, factor=0.3, random_state=SEED)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=SEED)

In [ ]:
# Visualize the dataset
plt.figure(figsize=(8, 6))
plt.scatter(X[:, 0], X[:, 1], c=y, cmap='viridis', edgecolors='k')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Dataset sintetico con due classi di punti NON linearmente separabili')
plt.show()

# Implementazione del modello
Nella prossima cella, partiamo da un modello di Logistic Regression (cioe' una rete neurale MLP senza hidden layers). 

In [ ]:
# Logistic Regression model
model = Sequential()

# Questo e' l'input layer
model.add(Input(shape=(2,)))

# Questo e' l'output layer
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

# Model training
Vediamo se riusciamo a separare le due classi con un modello di Logistic Regrassion.

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val))

# Analizziamo il processo di training
Analizziamo il grafico dell'errore sul training e validation set.

In [ ]:
history_dict = model.history.history
plt.plot(history_dict['loss'], label='Errore sul training set (Training Loss)')
plt.plot(history_dict.get('val_loss', []), label='Errore sul validation set (Validation Loss)', linestyle='dashed')
plt.xlabel('Epochs')
plt.ylabel('Errore (Loss)')
plt.legend()
plt.show()

# Visualizziamo il decision boundary sul training set

In [ ]:
weights, bias = model.layers[0].get_weights()
coefficients = [weights[0][0], weights[1][0], bias[0]]

# Calculate slope and intercept for the decision boundary line
slope = -coefficients[0] / coefficients[1]
intercept = -coefficients[2] / coefficients[1]

# Plot the data points and decision boundary line
plt.figure(figsize=(8, 6))
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, edgecolors='k', cmap='viridis')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')

# Plot the decision boundary line
x_min, x_max = X_train[:, 0].min(), X_train[:, 0].max()
y_min, y_max = X_train[:, 1].min(), X_train[:, 1].max()
plt.plot([x_min, x_max], [-1, 1], color='red', linestyle='--')

plt.title('Decision Boundary con una linea retta')
plt.show()

# Usiamo il modello addestrato sul test set

In [ ]:
_,test_accuracy = model.evaluate(X_test,y_test)

# Trasformiamo Logistic Regression in un modello MLP
Proviamo ora a risolvere il problema NON linearmente separabile con una rete neurale di tipo MLP. 
Per fare cio', modifichiamo il codice di Logistic Regression aggiungendo uno o piu' hidden layers.

In [ ]:
# MLP model
def create_model(hidden_layers=0, hidden_units=1):
    model = Sequential(name  = "mlp")
    # Questo e' l'input layer
    model.add(Input(shape=(2,)))
    
    # I seguenti sono gli hidden layers
    for layer in range(hidden_layers):
        model.add(Dense(hidden_units, activation='relu'))
    
    # Infine l'output layer
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
    print (model.summary())
    return model

# Configuriamo manualmente il modello MLP
Scegliamo manualmente il numero di hidden layers e hidden units (neuroni) con cui configurare il modello. Scegliamo anche per quante epoche addestrare il modello assegnando un valore alla variabile ```EPOCHS```.

In [ ]:
### Cambia questi valori manualmente ####
HIDDEN_LAYERS= 1
HIDDEN_UNITS = 4
EPOCHS=100
#########################################

model = create_model(hidden_layers=HIDDEN_LAYERS,hidden_units=HIDDEN_UNITS)

start_time = time.time()
model.fit(X_train, y_train, epochs=EPOCHS, validation_data=(X_val, y_val))
stop_time = time.time()

# Total training time
print("Total training time (sec): ", stop_time-start_time)

# Analizziamo il processo di training
Cosa possiamo cambiare nel modello (o nel numero di epoche) per ottenere un modello ottimale. Analizziamo il grafico dell'errore sul training e validation set.

In [ ]:
history_dict = model.history.history
plt.plot(history_dict['loss'], label='Errore sul training set (Training Loss)')
plt.plot(history_dict.get('val_loss', []), label='Errore sul validation set (Validation Loss)', linestyle='dashed')
plt.xlabel('Epochs')
plt.ylabel('Errore (Loss)')
plt.legend()
plt.show()

# Usiamo il modello addestrato sul test set

In [ ]:
_,test_accuracy = model.evaluate(X_test,y_test)

# Visualizziamo il decision boundary rispetto al training set
Vediamo quanto ha imparato il modello

In [ ]:
# Plot the decision boundary as a single curve
plt.figure(figsize=(8, 6))
h = .02  # Step size in the mesh
x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
grid = np.vstack([xx.ravel(), yy.ravel()]).T

# Predict probabilities for each point on the meshgrid
Z = model.predict(grid)
Z = Z.reshape(xx.shape)

# Plot the contour line representing the decision boundary (where probability is 0.5)
plt.contour(xx, yy, Z, levels=[0.5], colors='black')

# Plot the data points
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, edgecolors='k', cmap='viridis')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Decision Boundary con MLP')
plt.show()

# Early stopping
Invece di inserire il numero di epoche a mano, possiamo fare qualcosa di piu' furbo? **Early Stopping** e' un metodo per addestrare un modello per un numero ottimale di epoche.
Anche in questo caso va assegnato un parametro, chiamato ```PATIENCE``` (pazienza), che indica all'algoritmo quante epoche senza milgioramenti aspettare prima di fermare l'addestramento. Ad esempio, con ```PATIENCE=10```, se l'errore (loss) sul validation set raggiunge ```0.1``` e nelle successive 10 epoche non decresce, il processo viene fermato.

In [ ]:
### Cambia questi valori manualmente ###
HIDDEN_LAYERS= 1
HIDDEN_UNITS = 4

PATIENCE = 20
########################################


model = create_model(hidden_layers=HIDDEN_LAYERS,hidden_units=HIDDEN_UNITS)
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=PATIENCE, restore_best_weights=True)

start_time = time.time()
model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks= [early_stopping])
stop_time = time.time()

# Total training time
print("Total training time (sec): ", stop_time-start_time)

# Analizziamo il processo di training
Analizziamo di nuovo il grafico dell'errore sul training e validation set.

In [ ]:
history_dict = model.history.history
plt.plot(history_dict['loss'], label='Errore sul training set (Training Loss)')
plt.plot(history_dict.get('val_loss', []), label='Errore sul validation set (Validation Loss)', linestyle='dashed')
plt.xlabel('Epochs')
plt.ylabel('Errore (Loss)')
plt.legend()
plt.show()

# Usiamo il modello addestrato sul test set

In [ ]:
_,test_accuracy = model.evaluate(X_test,y_test)

# Visualizziamo il decision boundary rispetto al training set
Vediamo quanto ha imparato il modello

In [ ]:
# Plot the decision boundary as a single curve
plt.figure(figsize=(8, 6))
h = .02  # Step size in the mesh
x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
grid = np.vstack([xx.ravel(), yy.ravel()]).T

# Predict probabilities for each point on the meshgrid
Z = model.predict(grid)
Z = Z.reshape(xx.shape)

# Plot the contour line representing the decision boundary (where probability is 0.5)
plt.contour(xx, yy, Z, levels=[0.5], colors='black')

# Plot the data points
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, edgecolors='k', cmap='viridis')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Decision Boundary con MLP')
plt.show()

# Grid search
Il codice nella cella seguente esegue la configurazione automatica del modello MLP con la strategia ```grid search```.
Grid Search è una tecnica di ottimizzazione dei parametri dei modelli di Deep Learning (ma anche di modelli di Machine Learning) che consiste nella ricerca sistematica della combinazione ottimale di parametri (per esempio, hidden layers e hidden units (neuroni)), valutando le prestazioni di un modello su una griglia di possibili valori di questi parametri. La griglia dei parametri è  definita dal programmatore. 

Il tuo compito e' di inserire alcuni valori nella griglia sotto e di far partire l'addestramento del modello MLP.
Assegna anche un valore di ```PATIENCE``` assegnando un numero intero alla variabile ```PATIENCE```.

In [ ]:
# Create a KerasClassifier based on the create_model function
model = KerasClassifier(build_fn=create_model, batch_size=100, verbose=1)

### Inserisci alcuni valori interi separati da virgola tra le parentesi quadre ####
param_grid = {
    'hidden_layers' : [1,2],
    'hidden_units' : [4,8]
}
###########################################################

### Stabilisci il valore di PATIENCE (per esempio 10) ###
PATIENCE = 20
###########################################################

# Perform grid search with 5-fold cross-validation
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=2)

### Add early stopping
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=PATIENCE, restore_best_weights=True)

start_time = time.time()
grid_result = grid.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks= [early_stopping])
stop_time = time.time()

# Total training time
print("Total training time (sec): ", stop_time-start_time)
# Print the best parameters and corresponding accuracy
print("Best parameters found: ", grid_result.best_params_)
print("Best cross-validated accuracy: {:.2f}".format(grid_result.best_score_))

# Decision boundary
Vediamo se il nostro MLP e' in grado di distinguere le due classi.

In [ ]:
# Plot the decision boundary as a single curve
plt.figure(figsize=(8, 6))
h = .02  # Step size in the mesh
x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
grid = np.vstack([xx.ravel(), yy.ravel()]).T

# Predict probabilities for each point on the meshgrid
Z = best_model.predict(grid)
Z = Z.reshape(xx.shape)

# Plot the contour line representing the decision boundary (where probability is 0.5)
plt.contour(xx, yy, Z, levels=[0.5], colors='black')

# Plot the data points
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, edgecolors='k', cmap='viridis')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Decision Boundary con MLP')
plt.show()

# Usiamo il modello sul test set

In [ ]:
y_pred = np.squeeze(best_model.predict(X_test, batch_size=32) > 0.5)

print("F1 Score: ", f1_score(y_test,y_pred))

# Visualizziamo se e dove sbaglia sul test set

In [ ]:
# Plot the decision boundary as a single curve
plt.figure(figsize=(8, 6))
h = .02  # Step size in the mesh
x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
grid = np.vstack([xx.ravel(), yy.ravel()]).T

# Predict probabilities for each point on the meshgrid
Z = best_model.predict(grid)
Z = Z.reshape(xx.shape)

# Plot the contour line representing the decision boundary (where probability is 0.5)
plt.contour(xx, yy, Z, levels=[0.5], colors='black')

# Plot the data points
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, edgecolors='k', cmap='viridis')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Decision Boundary con MLP')
plt.show()